# Exercice 2 : Diffusion de particules

Dans cet exercice, vous allez simuler la diffusion de particules dans un milieu plan, comme dans le cas présenté dans la dernière vidéo.

Les grandes lignes de l'algorithme sont les suivantes :

## 1- Initialisation

Commencez par importer les librairies nécessaires.

In [102]:
# Importation des librairies
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
import matplotlib.colors as colors
import random

Il vous faut maintenant définir les dimensions du milieu dans lequel les particules vont évoluer.

In [6]:
# Définition des dimensions du milieu
H = 5 # hauteur du milieu en nombre de pixels
L = 8 # largeur du milieu en nombre de pixels

Choisissez également combien de temps durera la simulation (en nombre de pas de temps).

In [7]:
T = 10 # durée de la simulation en nombre de pas de temps

Définissons la source des particules. C'est la seule zone du milieu qui contient des particules au démarrage et toutes les particules quittant cette zone sont immédiatement remplacées.

Commencez par traiter le cas présenté dans la dernière vidéo : la  source est la ligne du bas du milieu.

Nous stockerons les points de la sources sous la forme d'un tableau contenant des tableaux de taille deux. Ces tableaux représenteront les coordonnées des points dans le milieu, la première coordonnée étant la hauteur à laquelle se situe la particule (en partant du haut) et la deuxième coordonnée étant la largeur à laquelle elle se situe (en partant de la gauche).

Créez votre source de particules.

In [15]:
source = [[H - 1, i] for i in range(L)]

Lors des pas de temps successifs les particules vont se déplacer, il nous faudra donc stoker les différentes états du milieu à tout instant. Pour cela, il nous faut créer le milieu dans lequel les particules vont évoluer et le remplir avec les particules présentes à t = 0. À l'instant t = 0 les seules particules présentes sont celles de la source.
l 
On représentera le milieu sous la forme d'une matrice de taille H * L rempli de VIDE partout où il n'y a pas de particules et PLEINE, là où il y en a.

In [9]:
VIDE = 0
PLEINE = 1
# Création du milieu
milieu = [[]] * H
for i in range(H) :
    milieu[i] = [VIDE] * L

# Placement des particules présentes 
for s in source :
    h, l = s
    milieu[h][l] = PLEINE

Il faut finalement créer le tableau qui stockera les états du milieu pour chaque instant.

In [45]:
stockage_milieu = [[-1]] * T

## 2- Simulation

Maintenant que nos variables sont initialisées, il nous faut simuler la diffusion. Cette partie n'implique pas l'affichage des résultats obtenus.

### 2-1- Choix des particules à déplacer

Il vous faut définir la fonction qui choisira les particules à déplacer. Ces particules doivent être choisies aléatoirement parmis la liste des particules présentes dans le milieu. Vous utiliserez pour cela la fonction sample de la bibliothèque random. 

Commencez par utiliser la fonction help pour obtenir les spécificaion de random.sample.

In [11]:
help(random.sample)

Help on method sample in module random:

sample(population, k) method of random.Random instance
    Chooses k unique random elements from a population sequence or set.
    
    Returns a new list containing elements from the population while
    leaving the original population unchanged.  The resulting list is
    in selection order so that all sub-slices will also be valid random
    samples.  This allows raffle winners (the sample) to be partitioned
    into grand prize and second place winners (the subslices).
    
    Members of the population need not be hashable or unique.  If the
    population contains repeats, then each occurrence is a possible
    selection in the sample.
    
    To choose a sample in a range of integers, use range as an argument.
    This is especially fast and space efficient for sampling from a
    large population:   sample(range(10000000), 60)



Il vous faudra choisir une certaine quantité de particules à sélectionner. Pour cela commencez par définir la proportion des particules qui doit être sélectionnée.

In [12]:
# proportion désigne la proportion des particules qui doivent être 
# sélectionnées pour être déplacées à chaque pas de temps.
# proportion est donc un flottant compris entre 0 et 1.
proportion = 0.5 

Créons maintenant la fonction choisir_particules.

In [20]:
def choisir_particules(milieu, proportion) :
    """
    Prérequis :
    milieu est une matrice contenant uniquement les valeurs VIDE et PLEINE.
    proportion est un flottant compris entre 0 et 1.
    
    Renvoie un tableau de coordonnées de particules présentes dans 
    milieu et choisies aléatoirement. 
    Ce tableau est de longueur :
    partie entière de proportion * ensemble_particules
    """
    # On commence par des tests sur la valeur de proportion.
    assert(proportion >= 0)
    assert(proportion <= 1)
    
    nombre_particules = 0
    ensemble_particules = []
    for h in range(H) :
        for l in range(L) :
            if milieu[h][l] == PLEINE :
                ensemble_particules.append([h, l])
                nombre_particules += 1
    # Remplissez la fonction
    k = int(len(ensemble_particules) * proportion)
    return random.sample(ensemble_particules, k)
    

Testez votre fonction sur l'ensemble de particules initial pour différentes valeurs de proportion. Testez notamment les cas limites.

In [23]:
# Testez choisir_particules
H = 3
L = 4
# Création du milieu
milieu = [[]] * H
for i in range(H) :
    milieu[i] = [VIDE] * L

# Placement des particules présentes 
for s in source :
    h, l = s
    milieu[h][l] = PLEINE
    
print(choisir_particules(milieu, 0))
print(choisir_particules(milieu, 0.5))
print(choisir_particules(milieu, 1))
print(choisir_particules(milieu, 0.33333))

[]
[[2, 0], [2, 2]]
[[2, 2], [2, 3], [2, 0], [2, 1]]
[[2, 0]]


### 2-2- Déterminer la destination des particules

Si une particule a été sélectionnée pour se déplacer, il faut choisir quelle sera sa destination. Pour rappel une particule doit aller aléatoirement dans une de ses destinations possibles (Est, Nord, Sud et Ouest à condition de ne pas sortir du milieu et de ne pas aller sur une autre particule).

Si aucune destination possible n'existe, une copie de particule est renvoyée.

Remplissez la fonction trouve_destination.

In [52]:
def trouve_destination(particule, milieu) :
    """
    Prérequis :
    particule est un tableau d'entiers positif à deux éléments et doit
    se situer dans le milieu.
    milieu est une matrice contenant exclusivent les valeurs VIDE et PLEINE.
    particule est un tableau à deux éléments contenant les coordonnées d'une case 
    de milieu contenant PLEINE.
    H et L sont des entiers strictement positifs définissant la taille du milieu
    
    Renvoie un tableau à deux éléments représentant la destination de particule.
    Cette destination est aléatoirement choisie parmis les destinations possibles 
    qui sont les cases du dessus, du dessous et des côtés de celles de particule
    si elles existent et ne sont pas occupées. 
    Si aucune destination possible n'existe, une copie de particule est renvoyée.
    """
    # Test d'une des préconditions
    [l, c] = particule
    assert(milieu[l][c] == PLEINE)
    
    ##############################################################################
    #                                 CONSEILS                                   #
    # Sélectionnez d'abord les destinations possibles et créez un tableau les    #
    # contenant. Veillez à bien vérifier que ces destinations ne soient pas      #
    # occupées et à ce qu'elles fassent bien partie du milieu. Vous n'avez       #
    # besoin pour cela que des coordonnées de particule.                         #
    # Vous pourez ensuite aléatoirement choisir un élément parmis les            #
    # destinations possibles. Veillez à bien gérer le cas où il n'y en a pas.    #
    ##############################################################################
    
    
    # Détermination des destinations possibles - À remplir
    
    # Choix et renvoi de la destination - À remplir
    # Détermination des destinations possibles - À remplir
    
    voisins_directs = [[l + 1, c], [l - 1, c], [l, c + 1], [l, c - 1]]
    destinations_possibles = []
    for v in voisins_directs :
        lv, cv = v
        if (lv >= 0) and (lv < H) and (cv >= 0) and (cv < L) and (milieu[lv][cv] == VIDE) :
            destinations_possibles.append(v.copy())
    
    # Choix et renvoi de la destination - À remplir
    if len(destinations_possibles) > 0 :
        return random.sample(destinations_possibles, 1)[0]
    else :
        return particule



Testez votre fonction à l'aide des cas suivants. Pour vous assurer que le résultat est correct, faites un dessin du milieu.
TESTEZ TOUS LES CAS LIMITES PRÉSENTÉS DANS LES PRÉCONDITIONS.

Une erreur à ce stade du code peut s'avérer très problématique pour la suite et difficile à détecter. Il est donc important d'effectuer correctement ce type de tests.

In [30]:
H = 7 
L = 3
milieu = [[PLEINE, VIDE  , VIDE  ],
          [VIDE  , PLEINE, VIDE  ],
          [PLEINE, PLEINE, PLEINE],
          [VIDE  , PLEINE, PLEINE],
          [VIDE  , VIDE  , VIDE  ],
          [VIDE  , PLEINE, VIDE  ],
          [VIDE  , VIDE  , VIDE  ]]
print(trouve_destination([0, 0], milieu, H, L)) # dans un coin 
print(trouve_destination([2, 1], milieu, H, L)) # coincée entre quatre particules
print(trouve_destination([2, 2], milieu, H, L)) # une seule destination possible
print(trouve_destination([5, 1], milieu, H, L)) # quatre destinations possibles

[0, 1]
[2, 1]
[1, 2]
[4, 1]


### 2-3- Déplacer une particule

Plaçons nous désormais dans le cas où nous devons déplacer une particule et en connaissons la destination. Il faut alors modifier milieu de la manière décrite dans les spécifications de la fonction deplace_particules.
    
Remplissez la fonction deplace_particule.

In [42]:
def deplace_particule(particule, destination) :
    """
    Prérequis : 
    particule et destination sont des particules, c'est à dire des tableaux d'entiers
    à deux éléments. destination n'appartient pas à ensemble_particules, définie dans 
    le code.
    
    Si particule et destination sont identiques, on ne fait rien.
    Sinon :
        On met VIDE à l'emplacement particule dans milieu et PLEINE à l'emplacement
        destination
        Si particule fait partie de la source, on met PLEINE à l'emplacement particule
        dans milieu.
        
    Ne renvoie rien.
    """
        
    # Remplissez la suite
    lp, cp = particule
    ld, cd = destination
    if particule != destination :
        assert(milieu[ld][cd] == VIDE)
        milieu[lp][cp] = VIDE
        milieu[ld][cd] = PLEINE
        if particule in source :
            milieu[lp][cp] = PLEINE
    else :
        pass

Testez votre fonction dans différents cas de figure.

In [44]:
H = 7 
L = 3
source = [[H - 1, i] for i in range(L)]
milieu = [[PLEINE, VIDE  , VIDE  ],
          [VIDE  , PLEINE, VIDE  ],
          [PLEINE, PLEINE, PLEINE],
          [VIDE  , PLEINE, PLEINE],
          [VIDE  , VIDE  , VIDE  ],
          [VIDE  , PLEINE, VIDE  ],
          [VIDE  , VIDE  , VIDE  ]]


# Test pour une particule hors de la source
particule = [1, 1]
destination = [0, 1]
deplace_particule(particule, destination)
print(milieu)

# Test pour une particule dans la source
particule = [6, 0]
destination = [5, 0]
deplace_particule(particule, destination)
print(milieu)

# D'autres idées ?
particule = [0, 1]
destination = [0, 2]
deplace_particule(particule, destination)
print(milieu)

[[1, 1, 0], [0, 0, 0], [1, 1, 1], [0, 1, 1], [0, 0, 0], [0, 1, 0], [0, 0, 0]]
[[1, 1, 0], [0, 0, 0], [1, 1, 1], [0, 1, 1], [0, 0, 0], [1, 1, 0], [1, 0, 0]]
[[1, 0, 1], [0, 0, 0], [1, 1, 1], [0, 1, 1], [0, 0, 0], [1, 1, 0], [1, 0, 0]]


### 2-4- Assemblage 

Vous avez défini un certain nombre de fonctions qui vous seront utiles pour coder la simulation entière. Il vous faut désormais tout assembler pour faire la simulation.
Pour rappel, l'algorithme est le suivant :

À vous de jouer ! Coder tout l'algorithme SAUF L'AFFICHAGE. 

SOYEZ TRÈS VIGILANT AUX PROBLÈMES D'ALIASING, NOTAMMENT DES MATRICES.

In [117]:
# Simulation

# Initialisation des jeux de données

H = 20
L = 50
proportion = 0.5
T = 150

source = [[H - 1, i] for i in range(L)] # Création du milieu
milieu = [[]] * H
for i in range(H) :
    milieu[i] = [VIDE] * L

# Placement des particules présentes 
for s in source :
    h, l = s
    milieu[h][l] = PLEINE
    
stockage_milieu = [[ligne.copy() for ligne in milieu]]

for t in range(1, T):
    
    # Choix des particules à déplacer pour ce tour
    particules_a_deplacer = choisir_particules(milieu, proportion)
    for particule in particules_a_deplacer :
        # Choix de la destination
        destination = trouve_destination(particule, milieu)

        # Déplacer ou ajouter la particule
        deplace_particule(particule, destination)
        
    # Stockage des résultats pour le pas de temps en cours 
    stockage_milieu.append([ligne.copy() for ligne in milieu])

Regardez si tout est normal sur la dernière image produite.

In [118]:
print(stockage_milieu[-1])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,

## 3- Affichage

Nous utiliserons ici une méthode d'affichage légèrement différente de celles utilisées précédemment, celle-ci ne demande pas d'adaptation des données.

Vous pouvez désormais tester votre simulation ! L'affichage peut prendre un peu de temps même si H, L et T sont modestes.

In [119]:
# Création du graphique
fig, ax = plt.subplots()
# Création de la palette de couleur
cmap = colors.LinearSegmentedColormap.from_list("",["green","orange"])

# Affichage 

# initialisation de la commande 
im = ax.imshow(stockage_milieu[0], cmap=cmap, vmin=0, vmax=1)


# fonction appelée pour l'initialisation des données à afficher
def init():
    im.set_data(stockage_milieu[0])
    return [im]

# fonction appelée pour l'actualisation des données à afficher
def animate(i):
    data_slice = stockage_milieu[i]
    im.set_data(data_slice)
    return [im]

# animation 
anim=animation.FuncAnimation(fig, animate, init_func=init,
                             frames=T, interval=30, blit=True,
                             repeat=False)
plt.close()
rc('animation', html='jshtml')
anim